In [4]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import json
#from scipy import stats
#import scipy.interpolate as interp
from pprint import pprint
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14

In [5]:
import FEMmodule as FEM

# Toy FEM - python version

This code is the Python replica of the matlab code used in *LAB01* of the course **Nonlinear Analysis of Aerospace Structures** held by Marco Morandini, Riccardo Vescovini at *Politecnico di Milano*